## Cài đặt thư viện pycryptodome

In [1]:
pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA256
import hashlib

## A. Xây dựng một module gồm các chức năng chính:

# 1. Phát sinh một khoá bí mật Ks của thuật toán AES

In [3]:
def generate_secret_key():
    return get_random_bytes(16)  # 16 bytes = 128 bits

## 2. Mã hoá tệp tin sử dụng thuật toán AES với khoá Ks

In [4]:
def encrypt_file(key, input_file, output_file):
    cipher = AES.new(key, AES.MODE_EAX)
    with open(input_file, 'rb') as f:
        plaintext = f.read()
        ciphertext, tag = cipher.encrypt_and_digest(plaintext)
    with open(output_file, 'wb') as f:
        f.write(cipher.nonce + ciphertext)

# 3. Giải mã tệp tin sử dụng thuật toán AES với khoá Ks        

In [5]:
def decrypt_file(key, input_file, output_file):
    with open(input_file, 'rb') as f:
        nonce, ciphertext = f.read(16), f.read()
        cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
        plaintext = cipher.decrypt(ciphertext)
    with open(output_file, 'wb') as f:
        f.write(plaintext)

# 4. Phát sinh một cặp khoá Kprivate và Kpublic của thuật toán RSA

In [6]:
def generate_rsa_key_pair():
    # Phát sinh cặp khóa RSA
    key = RSA.generate(2048)
    # Gán khóa phát sinh cho biến private_key 
    private_key = key
    # Tạo public key từ khóa đã phảt sinh
    public_key = key.publickey()
    return private_key, public_key

# 5. Mã hoá một chuỗi sử dụng thuật toán RSA sử dụng khoá Kpublic

In [7]:
def encrypt_string(text, public_key):
    cipher = PKCS1_OAEP.new(public_key)
    ciphertext = cipher.encrypt(text)
    return ciphertext

# 6. Giải mã một chuỗi sử dụng thuật toán RSA sử dụng khoá Kprivate

In [8]:
def decrypt_string(text, private_key):
    cipher = PKCS1_OAEP.new(private_key)
    decrypted_text = cipher.decrypt(text)
    return decrypted_text

# 7. Tính giá trị hash của một chuỗi sử dụng thuật toán SHA-1, SHA-256

In [9]:
# Tính giá trị hash của một chuỗi sử dụng thuật toán SHA-1
def hash_string_sha1(input_string):
    return hashlib.sha1(input_string.encode()).hexdigest()

# Tính giá trị hash của một chuỗi sử dụng thuật toán SHA-256
def hash_string_sha256(input_string):
    return hashlib.sha256(input_string.encode()).hexdigest()

# B. Xây dựng một ứng dụng để sử dụng các chức năng của module ở mục A

# 1. Người dùng mã hóa một tập tin

# Mã hóa file input.txt và ghi vào file encrypted.txt

In [10]:
input_file = 'input.txt'
encrypted_file = 'encrypted.txt'
# Phát sinh khóa bí mật 
secret_key = generate_secret_key() 
# Thực hiện mã hóa file input.txt và lưu vào file encrypted.txt
encrypt_file(secret_key, input_file, encrypted_file)
# Phát sinh private key và public key của thuật toán RSA
private_key, public_key = generate_rsa_key_pair()
# Mã hóa khóa bí mật bằng public key
Kx = encrypt_string(secret_key, public_key)
# Tính giá trị hash SHA-1 cho private key
HKprivate = hash_string_sha1(str(private_key))
print('Mã hoá hoàn tất')
print('Khoá Kprivate để giải mã: ',private_key)

Mã hoá hoàn tất
Khoá Kprivate để giải mã:  Private RSA key at 0x1B057978730


# 2. Người dùng giải mã một tập tin

# Giải mã file encrypted.txt và ghi vào file decrypted.txt

In [11]:
encrypted_file = 'encrypted.txt'
decrypted_file = 'decrypted.txt'
# Người dùng nhập khóa Kprivate
Kprivate = input('Nhập khoá Kprivate: ')
# Tính giá trị hash SHA-1 của Kprivate và so sánh với HKprivate, nếu không trùng thì giải mã thất bại
sha1_Kprivate = hash_string_sha1(Kprivate)

while sha1_Kprivate != HKprivate:
    Kprivate = input('Khoá Kprivate không hợp lệ, vui lòng nhập lại: ')
    sha1_Kprivate = hash_string_sha1(Kprivate)
# Giải mã chuỗi Kx để có được Ks 
Ks = decrypt_string(Kx, private_key)
# Dùng Ks giải mã tập tin 
decrypt_file(Ks, encrypted_file, decrypted_file)
print('Giải mã hoàn tất')

Nhập khoá Kprivate:  Private RSA key at 0x1B057978730


Giải mã hoàn tất
